# Conditional Generation of time_to_event using TabDDPM Architecture

In this notebook, we demonstrate how to build a conditional diffusion model that generates the `time_to_event` for a patient conditioned on both categorical and numerical features. This notebook follows the TabDDPM spirit by combining these feature types while inverting the typical supervised regression problem.

The steps include:

1. **Data Preprocessing:** Loading the unos liver transplant dataset, separating out the target (`time_to_event`), scaling numerical features, and one-hot encoding categorical features. All non-target features form the conditioning vector.
2. **Diffusion Setup:** Defining a cosine beta schedule and helper functions to add noise to the target.
3. **Model Definition:** Building a denoising network (an MLP) that takes as input the noisy target, a sinusoidal time embedding, and the fixed condition vector, and predicts the noise that was added.
4. **Training:** Using a DDPM-style training loop with an MSE loss between the predicted noise and the true noise.
5. **Sampling:** Implementing a reverse diffusion (using a simple DDIM sampler) that generates a plausible `time_to_event` given a condition.

Let's get started!

In [ ]:
import os
import json
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


## 1. Load and Preprocess Dataset

We assume a CSV file `unos_liver_transplant.csv` is available with these columns:

- **Numerical:** INIT_AGE, INIT_BMI_CALC, INIT_SERUM_SODIUM, INIT_SERUM_CREAT, INIT_INR, INIT_BILIRUBIN, INIT_ALBUMIN, INIT_MELD, time_to_event
- **Categorical:** GENDER, ABO, Etiology, Ethnicity, diab_group_labeled, Encephalopathy_Status, Ascites_Status, INIT_DIALYSIS_PRIOR_WEEK

We separate `time_to_event` as the target and combine the remaining numerical features (after scaling) with one-hot encoded categorical features to form our condition vector.

In [ ]:
# Load dataset (update the path as needed)
data_path = 'unos_liver_transplant.csv'
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
else:
    # For demonstration, create a dummy dataframe with similar structure
    num_data = {
        'INIT_AGE': np.random.randint(20, 80, 1000),
        'INIT_BMI_CALC': np.random.uniform(18, 35, 1000),
        'INIT_SERUM_SODIUM': np.random.uniform(120, 145, 1000),
        'INIT_SERUM_CREAT': np.random.uniform(0.5, 2.0, 1000),
        'INIT_INR': np.random.uniform(0.8, 2.5, 1000),
        'INIT_BILIRUBIN': np.random.uniform(0.5, 5.0, 1000),
        'INIT_ALBUMIN': np.random.uniform(2.5, 4.5, 1000),
        'INIT_MELD': np.random.uniform(6, 40, 1000),
        'time_to_event': np.random.uniform(0, 3650, 1000)
    }
    cat_data = {
        'GENDER': np.random.choice(['M', 'F'], 1000),
        'ABO': np.random.choice(['A', 'B', 'O'], 1000),
        'Etiology': np.random.choice(['Alcohol', 'Autoimmune', 'Cryptogenic', 'NASH'], 1000),
        'Ethnicity': np.random.choice(['White', 'Black/African American', 'Hispanic/Latino'], 1000),
        'diab_group_labeled': np.random.choice(['Diabetes', 'No Diabetes'], 1000),
        'Encephalopathy_Status': np.random.choice(['Grade 1-2', 'Grade 3-4', 'NaN'], 1000),
        'Ascites_Status': np.random.choice(['Slight', 'Moderate', 'NaN'], 1000),
        'INIT_DIALYSIS_PRIOR_WEEK': np.random.choice([0, 1], 1000)
    }
    df = pd.DataFrame({**num_data, **cat_data})
    print('Dummy dataset created')

# Specify columns
target_col = 'time_to_event'
numerical_cols = [
    'INIT_AGE', 'INIT_BMI_CALC', 'INIT_SERUM_SODIUM',
    'INIT_SERUM_CREAT', 'INIT_INR', 'INIT_BILIRUBIN',
    'INIT_ALBUMIN', 'INIT_MELD'
]
categorical_cols = [
    'GENDER', 'ABO', 'Etiology', 'Ethnicity',
    'diab_group_labeled', 'Encephalopathy_Status',
    'Ascites_Status', 'INIT_DIALYSIS_PRIOR_WEEK'
]

# Extract target (time_to_event)
y = df[target_col].values.astype(np.float32).reshape(-1, 1)

# Process numerical features: scale them (simple min-max scaling)
num_features = df[numerical_cols].values.astype(np.float32)
num_min = num_features.min(axis=0)
num_max = num_features.max(axis=0)
num_scaled = (num_features - num_min) / (num_max - num_min + 1e-8)

# Process categorical features using one-hot encoding
cat_df = pd.get_dummies(df[categorical_cols], drop_first=True)
cat_features = cat_df.values.astype(np.float32)

# Combine conditioning features: numerical (excluding target) + categorical
condition = np.hstack([num_scaled, cat_features])
print('Condition shape:', condition.shape)
print('Target shape:', y.shape)


## 2. Create PyTorch Dataset and DataLoader

We create a simple Dataset that returns a tuple of `(target, condition)` for each patient.

In [ ]:
class TransplantDataset(Dataset):
    def __init__(self, condition, target):
        self.condition = torch.tensor(condition, dtype=torch.float32)
        self.target = torch.tensor(target, dtype=torch.float32)

    def __len__(self):
        return len(self.target)
    
    def __getitem__(self, idx):
        return self.target[idx], self.condition[idx]

dataset = TransplantDataset(condition, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
print(f'Dataset size: {len(dataset)}')


## 3. Diffusion Hyperparameters and Helper Functions

We use a cosine schedule to compute the beta values and then derive the corresponding alphas and cumulative products. The function `q_sample` adds noise to the target value based on a given timestep.

In [ ]:
T = 1000  # total diffusion timesteps

def get_beta_schedule_cosine(T):
    steps = T
    s = 0.008
    t = np.linspace(0, steps, steps+1) / steps
    alphas_cumprod = np.cos((t + s) / (1 + s) * np.pi / 2) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = []
    for i in range(steps):
        beta = 1 - alphas_cumprod[i+1] / alphas_cumprod[i]
        betas.append(min(beta, 0.999))
    return np.array(betas, dtype=np.float32)

betas = get_beta_schedule_cosine(T)
alphas = 1 - betas
alphas_cumprod = np.cumprod(alphas)

def q_sample(x0, t, noise):
    """
    Diffuse the data at timestep t
    x0: original data (batch, 1)
    t: timestep tensor (batch,)
    noise: noise tensor
    """
    # Convert cumulative products to tensors
    sqrt_alphas_cumprod = torch.tensor(np.sqrt(alphas_cumprod), dtype=torch.float32, device=x0.device)
    sqrt_one_minus_alphas_cumprod = torch.tensor(np.sqrt(1 - alphas_cumprod), dtype=torch.float32, device=x0.device)
    a = sqrt_alphas_cumprod[t].unsqueeze(1)
    b = sqrt_one_minus_alphas_cumprod[t].unsqueeze(1)
    return a * x0 + b * noise

def sample_t(batch_size):
    return torch.randint(0, T, (batch_size,), device=device).long()


## 4. Time Embedding

We define a sinusoidal time embedding function to inject information about the diffusion timestep into the model.

In [ ]:
def get_timestep_embedding(timesteps, embedding_dim):
    """
    Returns a sinusoidal embedding for t
    """
    half_dim = embedding_dim // 2
    emb = np.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, device=timesteps.device, dtype=torch.float32) * -emb)
    emb = timesteps.float().unsqueeze(1) * emb.unsqueeze(0)
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:
        emb = torch.cat([emb, torch.zeros(timesteps.size(0), 1, device=timesteps.device)], dim=1)
    return emb

embedding_dim = 32  # dimension of time embedding


## 5. Define the Denoising Network (MLP)

The network takes as input the concatenation of:
- A noisy version of the target (shape: [batch, 1])
- The condition vector (combined numerical and categorical features)
- A time embedding for the current diffusion timestep

It outputs a prediction of the noise that was added to the target.

In [ ]:
class DenoiseMLP(nn.Module):
    def __init__(self, condition_dim, time_emb_dim, hidden_dim=128):
        super(DenoiseMLP, self).__init__()
        input_dim = 1 + condition_dim + time_emb_dim  # 1 for target
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)  # output: predicted noise
        )

    def forward(self, x_target_noisy, condition, t):
        # Get time embedding
        t_emb = get_timestep_embedding(t, embedding_dim)
        # Concatenate along feature dimension
        x = torch.cat([x_target_noisy, condition, t_emb], dim=1)
        return self.net(x)

# Determine condition dimension
condition_dim = condition.shape[1]
model = DenoiseMLP(condition_dim, embedding_dim).to(device)
print(model)


## 6. Training Loop

We now train the model to predict the noise added to the target value. For each batch:

1. Sample a random diffusion timestep `t` for every example.
2. Sample Gaussian noise and add it to the true target using the `q_sample` function.
3. Feed the noisy target, condition vector, and timestep into the network to predict the noise.
4. Compute the MSE loss between the predicted and true noise and backpropagate.


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 5  # for demonstration purposes

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for x_target, cond in dataloader:
        x_target = x_target.to(device)  # shape: (batch, 1)
        cond = cond.to(device)          # shape: (batch, condition_dim)
        batch_size = x_target.size(0)
        t = sample_t(batch_size)        # random timesteps for each sample
        noise = torch.randn_like(x_target)
        x_target_noisy = q_sample(x_target, t, noise)
        
        optimizer.zero_grad()
        noise_pred = model(x_target_noisy, cond, t)
        loss = nn.MSELoss()(noise_pred, noise)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * batch_size
    
    epoch_loss /= len(dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


## 7. Sampling: Generate time_to_event for a Given Condition

Next, we implement a reverse diffusion (sampling) procedure. Starting from random noise, we iterate through the timesteps (using a DDIM-style update with eta=0 for determinism) to generate a sample for `time_to_event` given a condition vector.

In [ ]:
def p_sample(model, x, cond, t):
    """
    One reverse diffusion step.
    x: current noisy sample (batch, 1)
    cond: condition vector (batch, condition_dim)
    t: current timestep tensor (batch,)
    """
    with torch.no_grad():
        noise_pred = model(x, cond, t)
        # Compute predicted x0
        sqrt_alphas_cumprod_t = torch.tensor(np.sqrt(alphas_cumprod), dtype=torch.float32, device=x.device)[t].unsqueeze(1)
        sqrt_one_minus_alphas_cumprod_t = torch.tensor(np.sqrt(1 - alphas_cumprod), dtype=torch.float32, device=x.device)[t].unsqueeze(1)
        x0_pred = (x - sqrt_one_minus_alphas_cumprod_t * noise_pred) / sqrt_alphas_cumprod_t
        
        # DDIM update (eta=0 for deterministic sampling)
        t_next = t - 1
        t_next = torch.clamp(t_next, min=0)
        sqrt_alphas_cumprod_next = torch.tensor(np.sqrt(alphas_cumprod), dtype=torch.float32, device=x.device)[t_next].unsqueeze(1)
        sqrt_one_minus_alphas_cumprod_next = torch.tensor(np.sqrt(1 - alphas_cumprod), dtype=torch.float32, device=x.device)[t_next].unsqueeze(1)
        x_next = sqrt_alphas_cumprod_next * x0_pred + sqrt_one_minus_alphas_cumprod_next * noise_pred
        return x_next

def sample_conditional(model, cond, num_steps=T):
    model.eval()
    batch_size = cond.size(0)
    # Start from pure noise for the target
    x = torch.randn((batch_size, 1), device=device)
    for i in reversed(range(num_steps)):
        t = torch.full((batch_size,), i, device=device, dtype=torch.long)
        x = p_sample(model, x, cond, t)
    return x

# Example: sample for 5 random conditions from the dataset
sample_conditions = torch.tensor(condition[:5], dtype=torch.float32).to(device)
generated_time = sample_conditional(model, sample_conditions)
print('Generated time_to_event:', generated_time.cpu().numpy())


## Conclusion

We have now built a conditional diffusion model (following the TabDDPM idea of combining categorical and numerical features as conditions) that learns to generate the `time_to_event` for a patient given their full feature profile. This notebook illustrated the entire process from data preprocessing, model design, training, and sampling.

Feel free to modify hyperparameters, experiment with the model architecture, or extend the diffusion process as needed for your application.